This notebook is adapted from [a lesson from the 2017 KIPAC/StatisticalMethods course](https://github.com/KIPAC/StatisticalMethods/blob/2017winter/chunks/generative_models.ipynb), (c) 2017 Adam Mantz and Phil Marshall. The GPLv2 license applies.

# Missing Information and Selection Effects

Goals:
* Incorporate models for data selection into our toolkit
* Understand when selection effects are ignorable, and when they must be accounted for

**What does "missing information" mean?**

In physics, we're used to the idea that we never have complete information about a system.

Trivial example: non-zero measurement errors mean that we're missing some information, namely the true value of whatever we've measured. We deal with this by incorporating that fact into our model, via the sampling distribution.

Hierachical models tend to be full of such unobservable parameters, including things like group membership.

**Key messages**

1. No data set is perfectly complete (especially in astronomy!)
2. It's our job to know whether that incompleteness can be ignored for the purpose of our inference
3. If not, we need to model it appropriately and marginalize over our ignorance

**More missingness mechanisms**

Two more ways that data can be missing are extremely common in astrophysics, and especially in surveys. In statistics, these are called **censoring** and **truncation**.

These are related (though not one-to-one) with the astronomical terms **Malmquist bias** and **Eddington bias**.

**Censoring**: a given data point (astronomical source) is known to exist, but a relevant measurement for it is not available.

This refers both to completely absent measurements and upper limits/non-detections, although in principle the latter case still provides us with a sampling distribution.

**Truncation**: not only are measurements missing, but the total number of sources that *should* be in the data set is unknown.

In other words, the lack of a measurement means that we don't even know about a particular source's existence.

**Malmquist bias** refers to the fact that flux-limited surveys have an effective *luminosity* limit for detection that rises with distance (redshift). Thus, the sample of measured luminosities is not representative of the whole population.

<table>
    <tr>
        <td><a href="https://commons.wikimedia.org/wiki/File:Bias2.png"><img src="../graphics/missing_malquist.png" width=100%></a></td>
    </tr>
</table>

Image credit: Wikimedia Commons user Galaxy1F10 (public domain)

**Eddington bias** refers to the effect of noise or scatter on a luminosity function, $N(<L)$, the number of sources in some population less luminous than $L$.

Because the true $N(<L)$ is usually steeply decreasing in practice, and extends below the survey flux limit, scatter in measurements of $L$ can have a big impact on the measured luminosity function.

This is a histogram rather than $N(<L)$, but you get the idea.

<table>
    <tr>
        <td><img src="../graphics/missing_eddington.png" width=100%></td>
    </tr>
</table>

The terms Malmquist and Eddington bias were coined in relatively specific contexts. Usually, it's more accurate to say that a given data set is impacted by the selection procedure.

**Example**

Consider the (real) case of a flux-limited galaxy cluster survey. Cluster luminosities scale with mass, and the mass function (hence also the luminosity function) is steeply decreasing. The number as a function of mass and redshift, and the luminosity-mass relation, are both of interest.

<img src="../graphics/missing_RASS_zL.png" width=50%>
Complilation of ROSAT All-Sky Survey cluster detections

<table>
    <tr>
        <td><img src="../graphics/missing_expn_full.png" width=90%></td>
        <td></td>
        <td><img src="../graphics/missing_expn_trun.png" width=90%></td>
    </tr>
</table>

Fictional luminosity-mass data, applying a threshold for detection

Above 2 slides' image credits: A. Mantz ([MNRAS, 406:1773, 2010](http://adsabs.harvard.edu/abs/2010MNRAS.406.1773M))

**Coping with missing information**

Ad hoc approaches abound, but are usually nonsense. You might hear the terms "debiasing" or "deboosting" in this context.

Far better is to include the "selection" process that determines which data are observed and which are not in our generative model. This may involve expanding the model to include things like undetected sources.

**Formally Modelling Missing Information**

Adopting notation from Gelman (2004):
* $y_\mathrm{obs}$ and $y_\mathrm{mis}$ are the observed and unobserved data, and $y=y_\mathrm{obs}\cup y_\mathrm{mis}$
* $I$ is a vector of indicator variables (0 or 1) telling us whether a given y is observed or not
* $\theta$ is the set of parameters needed to model a completely observed data set
* $\phi$ are any additional parameters needed to model the selection process

We'll assume that $\theta$ and $\phi$ can always be separated.

The likelihood associated with a complete data set would be just

$P(y|\theta)$

For our partially missing data set, this needs to also account for the inclusion parameters, $I$

$P(y,I|\theta,\phi) = P(y|\theta)\,P(I|\phi,y)$

In other words, inclusion is part of the observed data vector.

Expanding out the $y$s,

$P(y_\mathrm{obs},y_\mathrm{mis},I|\theta,\phi) = P(y_\mathrm{obs},y_\mathrm{mis}|\theta)\,P(I|\phi,y_\mathrm{obs},y_\mathrm{mis})$

This isn't yet a likelihood for the *observed* data, however. For that we need to marginalize over the  $y_\mathrm{mis}$.

$P(y_\mathrm{obs},I|\theta,\phi) = \int dy_\mathrm{mis} \, P(y_\mathrm{obs},y_\mathrm{mis}|\theta)\,P(I|\phi,y_\mathrm{obs},y_\mathrm{mis})$

Note that we no longer have a clean separation between data and parameters in the generative model sense.

Thinking of drawing a PGM, the $y$ nodes can be fixed by observation *or* nuisance parameters, depending on the corresponding element of $I$.

**When can we ignore selection?**

Consider the likelihood in this form

$P(y_\mathrm{obs},I|\theta,\phi) = \int dy_\mathrm{mis} \, P(y_\mathrm{obs},y_\mathrm{mis}|\theta)\,P(I|\phi,y_\mathrm{obs},y_\mathrm{mis})$

We can get away with ignoring the selection process if the posterior for the parameters of interest $P(\theta|y_\mathrm{obs},I)$ is equivalent to simply $P(\theta|y_\mathrm{obs})$.

$P(\theta|y_\mathrm{obs},I)$

$= \int d\phi\int dy_\mathrm{mis} \, P(y_\mathrm{obs},y_\mathrm{mis}|\theta) \, P(I|\phi,y_\mathrm{obs},y_\mathrm{mis}) \, P(\theta,\phi)$

$= P(\theta|y_\mathrm{obs})$ ?

This requires two things to be true:

1. Selection doesn't depend on unobserved data: $P(I|\phi,y_\mathrm{obs},y_\mathrm{mis}) = P(I|\phi,y_\mathrm{obs})$
2. Priors for the interesting ($\theta$) and selection-related ($\phi$) parameters are independent: $P(\theta,\phi)=P(\theta)P(\phi)$

**Example:** black hole $M$-$\sigma$ relation

Imagine we're fitting the relation between the central black hole mass and bulge stellar velocity dispersion for galaxies.

<img src="../graphics/missing_Msigma.jpg" width=50%>

Image credit: Msigma at the English Language Wikipedia (Creative Commons Attribution-Share Alike 3.0 Unported)

To start with, we'll assume a complete data set. Then the generative model needs
* true values of $\sigma$ for the $N$ galaxies
* true values of $M$ for each galaxy, determined by a mean relation and scatter, parametrized by $\theta$
* sampling distributions for $M$ and $\sigma$, which we'll assume are independent
* prior distributions for $\sigma$ and $\theta$

$P(\hat{\sigma},\hat{M}|\sigma,M,\theta)= \prod_{k=1}^N P(M_k|\sigma_k,\theta)\,P(\hat{M}_k|M_k)\,P(\hat{\sigma}_k|\sigma_k)$

<img src="../graphics/missing_pgm0.png" width=70%>

Now imagine (realistically) that we don't have measurements for all the black hole masses.

The data need to be augmented by the inclusion vector, $I$, which implicitly includes the number of observed $M$'s, $N_\mathrm{obs}$.

Dividing the sample into **fully** and **partially** observed galaxies, the likelihood becomes

${N \choose N_\mathrm{full}} \prod_{k\,\in\,\mathrm{full}} P(M_k|\sigma_k,\theta)\,P(\hat{M}_k|M_k)\,P(\hat{\sigma}_k|\sigma_k)\,P(I_k=1|\bullet,\phi)$

$\times\prod_{k\,\in\,\mathrm{part}} \int d\hat{M}_k\, P(M_k|\sigma_k,\theta)\,P(\hat{M}_k|M_k)\,P(\hat{\sigma}_k|\sigma_k)\,P(I_k=0|\bullet,\phi)$

where $\phi$ are additional parameters related to selection, and $\bullet$ can in principle include any of $M_k$, $\sigma_k$, $\hat{M}_k$ and $\hat{\sigma}_k$.

Note that a binomial term, ${N \choose N_\mathrm{full}}$ has sneakily appeared.

The reason for this is subtle, and has to do with the statistical concept of *exchangeability* (a priori equivalence of data points).

As we've set things up, the fully observed data are exchangeable with one another, as are the partially observed data, but the the full data set is not, by virtue of containing these two classes.

It helps to think in terms of the generative model here. Namely, because the order of data points holds no meaning for us, the binomial term is there to reflect the number of ways we might generate completely equivalent (except for the ordering) data sets.

In other words, $P(I|\ldots)$ shouldn't actually give us the likelihood of a specificly ordered inclusion vector, but instead the likelihood that $I$ has the observed number of fully observed data points in it ($N_\mathrm{obs}$), along with any dependence on $(\bullet,\phi)$.

**Example: $M$-$\sigma$ data missing at random**

Consider this simple inclusion model: we tried to obtain measurements of $M$ for every galaxy, but that these attempts were only successful with probability $\phi$ (thanks to, e.g., weather).

<img src="../graphics/missing_pgm1.png" width=70%>

(Remember, $I_k$ tells us whether $\hat{M}_k$ is observed.)

**Is selection independent of unobserved data ($\hat{M}_k$)?**

Yes.

$P(I_k=1|\phi,\ldots) = P(I_k=1|\phi) = \phi$

$P(I_k=0|\phi,\ldots) = P(I_k=0|\phi) = 1-\phi$

**Are the priors for $\theta$ and $\phi$ independent?**

Yes, at least as drawn in the PGM.

Intuitively in this case, when data are missing at random, the selection effects are ignorable.

**Exercise**

Simplify the likelihood for the missing-at-random case, specifically taking note of the terms involving selection (i.e. $N_\mathrm{full}$ and $\phi$). Do you recognize the function that these terms form?

**Exercise**

Given the same basic problem (the $M$-$\sigma$ relation), for which of the PGMs below are selection effects ignorable? Sketch a cartoon of the complete data set and the effects of selection for each case. Do the conclusions about ignorability make intuitive sense?

<img src="../graphics/missing_pgm2.png" width=50%>

<img src="../graphics/missing_pgm3.png" width=50%>

<img src="../graphics/missing_pgm4.png" width=50%>

**Bonus exercise: galaxy cluster scaling relations**

This is a more general version of a regression where selection plays a role. It builds off of the galaxy cluster example illustrated above. This is a long exercise, but worth thinking through later on.

Our generative model will include
* true values of mass ($M$) for the $N$ clusters in some search volume
* true values of luminosity ($L$) and temperature ($T$) for each cluster, determined by mean relations and joint intrinsic scatter, all parametrized by $\theta$ (note that in this example, $M$ is the independent variable of the scaling relation, not the dependent one)
* sampling distributions for $L$, $T$ and $M$, which may not be independent (leading to measured values $\hat{L}$, $\hat{T}$, $\hat{M}$)
* prior distributions for $M$ and $\theta$
* selection-related parameter(s) $\phi$

If it helps to visualize things, you can assume the sampling distributions and the intrinsic scatter are multivariate normal. This isn't strictly necessary, though.

In this exercise, assume that only clusters with "observed luminosity" (which really means number of detected photons) over some threshold, $\hat{L}\geq L_\mathrm{th}$, are included in the sample. We have complete information for $N_\mathrm{obs}$ detected clusters, but no information for the remaining clusters, nor do we know their total number, $N$.

A good place to start is to sketch a cartoon of the scaling relations before/after selection, as above, draw the PGM, and perhaps write out an expression for the likelihood (including its potential conditional dependences).

We're interested in whether selection effects can be ignored for inferences about (1) the marginal $L$-$M$ relation, and (2) the marginal $T$-$M$ relation. Consider these questions for these increasingly complex models:
1. Assume that the intrinsic scatter in $L,T|M$ is uncorrelated and the measurement scatter in $\hat{L},\hat{T},\hat{M}|L,T,M$ is also uncorrelated
2. Allow the intrinsic scatter (only) to be correlated
3. Allow the measurement scatter to be correlated
4. Allow either to be correlated

Hint: it will probably help to refer to the PGM/likelihood, the rules for ignorability, and especially your intuition at each stage.

Hint: a great way to gain some insight is to simulate complete and truncated data sets. For example, to see the effect of intrinsic correlation for (2) above, you could use the (realistic) model:
* $P(M)\propto M^{-3}$
* $\left({\log L}\atop{\log T}\right) \sim \mathrm{Normal}\left[\alpha\,\log M, \Sigma\right]$
* $\alpha = \left({4/3}\atop{2/3}\right)$
* $\Sigma_{11}=0.4^2$, $\Sigma_{22}=0.15^2$, $\Sigma_{12}$ determined by a correlation coefficient you can vary